# Vectores de Palabras

La idea de que dos palabras pueden ser más o menos similares es crucial para muchas tareas de NLP.

Esta idea de similitud es más fácil de manejar si representamos las palabras como un vectores en lugar de enteros. Podemos decidir qué rasgos/características tenemos en cuenta, y cuántas dimensiones tienen nuestros vectores. Podemos diseñarlos a mano. Podemos calcularlos automáticamente.

En lingüística, en Semántica Distribucional, está claro que si dos palabras pueden utilizarse de manera similar es porque tienen significados parecidos.

> *You shall know a word by the company it keeps.* John R. Firth

> *The meaning of a word is its use in the language (…) One cannot guess how a word functions. One has to look at its use, and learn from that.* L. Wittgenstein 

Dos palabras son similares si aparecen en contextos similares.


Los modelos de espacio vectorial ([*vector space models*](https://en.wikipedia.org/wiki/Vector_space_model)) permiten representar palabras o términos dentro de un espacio vectorial continuo, de manera que las palabras que son similares desde el punto semántico se situan en puntos cercanos dentro de ese espacio común.

![](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2017/06/06062705/Word-Vectors.png)


## `word2vec`

(Mikolov, 2013) describe [word2vec](https://en.wikipedia.org/wiki/Word2vec), un algoritmo no supervisado para calcular el significado de palabras como representaciones vectoriales densas, como listas de números reales.

[GloVe](https://nlp.stanford.edu/projects/glove/), en 2014, propone un mecanismo similar.

Estos sistemas aprenden el significado de las palabras, y parecen capturar cierta dirección semántica y morfo-sintáctica.

    París - France + España ≈ Madrid
    comimos - comer + andar ≈ anduvimos


In [ ]:
import warnings
warnings.filterwarnings(action="ignore")

## Similitud semántica entre palabras, frases y documentos

spaCy permite [calcular la similitud semántica](https://spacy.io/usage/vectors-similarity) entre cualquier par de objetos de tipo `Doc`, `Span` o `Token`. 

Ojo, La similitud semántica es un concepto algo subjetivo, pero en este caso se puede entender como la probabilidad de que dos palabras aparezcan en los mismos contextos.

In [ ]:
import spacy

nlp_en = spacy.load("en_core_web_md")
nlp_es = spacy.load("es_core_news_md")

In [ ]:
# analizamos algunas colocaciones en inglés
token1, _, token2 = nlp_en("cats and dogs")
token3, _, token4 = nlp_en("research and development")

# medimos la similitud semántica entre algunos pares
print(f"{token1} vs {token2}: {token1.similarity(token2)}")
print(f"{token3} vs {token4}: {token3.similarity(token4)}")
print(f"{token1} vs {token4}: {token1.similarity(token4)}")

In [ ]:
# ¿qué tal funciona en español?
token1, _, token2 = nlp_es("perros y gatos")
token3, _, token4 = nlp_es("investigación y desarrollo")

# medimos la similitud semántica entre algunos pares
print(f"{token1} vs {token2}: {token1.similarity(token2)}")
print(f"{token3} vs {token4}: {token3.similarity(token4)}")
print(f"{token1} vs {token4}: {token1.similarity(token4)}")

In [ ]:
print(f"{token1} tiene {token1.vector.shape} dimensiones")
print(token1.vector)

# Ejemplo de `word2vec` con `gensim`

En la siguiente celda, importamos las librerías necesarias y configuramos los mensajes de los logs.

In [ ]:
import gensim, logging, os
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

model = gensim.models.Word2Vec.load("/home/victor/w2v/eswiki-300.w2v")

print(f"Modelo cargado con un vocabulario de {model.corpus_count} términos diferentes.")

Cada término del vocabulario está representado como un vector con 300 dimensiones. Podemos acceder al vector de un término concreto, aunque probablemente no nos diga mucho, salvo que contienen números muy pequeños :-/

In [ ]:
print(model["azul"])
print(model["verde"])
print(model["clorofila"])

Este mismo objeto `model` permite acceder a una serie de funcionalidades ya implementadas que nos van a permitir evaluar formal e informalmente el modelo. Por el momento, nos contentamos con los segundo: vamos a revisar visualmente los significados que nuestro modelo ha aprendido por su cuenta. 

Podemos calcular la similitud semántica entre dos términos usando el método `similarity`, que nos devuelve un número entre 0 y 1:

In [ ]:
print("hombre - mujer", model.wv.similarity("hombre", "mujer"))

print("perro - gato", model.wv.similarity("perro", "gato"))

print("gato - periódico", model.wv.similarity("gato", "periódico"))

print("febrero - azul", model.wv.similarity("febrero", "azul"))

Podemos seleccionar el término que no encaja a partir de una determinada lista de términos usando el método `doesnt_match`:

In [ ]:
lista1 = "madrid barcelona gonzález washington".split()
print("en la lista", " ".join(lista1), "sobra:", model.wv.doesnt_match(lista1))

lista2 = "psoe pp ciu ronaldo".split()
print("en la lista", " ".join(lista2), "sobra:", model.wv.doesnt_match(lista2))

lista3 = "publicaron declararon soy negaron".split()
print("en la lista", " ".join(lista3), "sobra:", model.wv.doesnt_match(lista3))

lista4 = "homero saturno cervantes shakespeare cela".split()
print("en la lista", " ".join(lista4), "sobra:", model.wv.doesnt_match(lista4))

lista5 = "madrid barcelona alpedrete marsella".split()
print("en la lista", " ".join(lista5), "sobra:", model.wv.doesnt_match(lista5))

Podemos buscar los términos más similares usando el método `most_similar` de nuestro modelo:

In [ ]:
terminos = "azul madrid bmw bici 2018 rock google psoe jay-z xiaomi rajoy brexit saturno césar lazio".split()

for t in terminos:
    print(t, "==>", model.wv.most_similar(t), "\n")

Con el mismo método `most_similar` podemos combinar vectores de palabras tratando de jugar con los rasgos semánticos de cada una de ellas para descubrir nuevas relaciones.

In [ ]:
print("mujer que ejerce la autoridad en una alcaldía ==> alcalde + mujer - hombre")
most_similar = model.wv.most_similar(positive=["alcalde", "mujer"], negative=["hombre"], topn=3)
for item in most_similar:
    print(item)
    
print("mujer especializada en alguna terapia de la medicina ==> doctor + mujer - hombre")
most_similar = model.wv.most_similar(positive=["doctor", "mujer"], negative=["hombre"], topn=3)
for item in most_similar:
    print(item)

print("monarca soberano ==> reina + hombre - mujer")    
most_similar = model.wv.most_similar(positive=["reina", "hombre"], negative=["mujer"], topn=3)
for item in most_similar:
    print(item)
    
print("capital de Alemania ==> moscú + alemania - rusia")
most_similar = model.wv.most_similar(positive=["moscú", "alemania"], negative=["rusia"], topn=3)
for item in most_similar:
    print(item)

print("presidente de Francia ==> obama + francia - eeuu")
most_similar = model.wv.most_similar(positive=["obama", "francia"], negative=["eeuu"], topn=3)
for item in most_similar:
    print(item)

## `fastText`

`word2vec` y `GloVe` tienen un problema: no saben cómo manejar palabras desconocidas (*OOV words*).

`fastText` (Bojanowski et al., 2016) describe un método para aprender vectores, no de palabras, sino de n-gramas o secuencias de caracteres (*subword information*).

Es capaz de manejar erratas y palabras raras y[está disponible en 157 lenguas del mundo](https://fasttext.cc/docs/en/crawl-vectors.html#models).

Pero, estos modelos tienen una limitación importante: cada palabra se codifica como un único vector.

Y el lenguaje natural es ambiguo, a varios niveles. De hecho, existen fenómenos como la polisemia y la homografía.

- *banco*: de sentar, de crédito, de peces, de arena
- *cubo*: de agua, operación matemática, figura geométrica
- *planta*: arbusto, fábrica, del pie
- *bajo*: nombre, adjetivo, preposición

## Barrio Sésamo al Rescate

![](http://2kceni4cuv0n2yx0s3ezy0bk-wpengine.netdna-ssl.com/wp-content/uploads/2019/07/Lb8cuv8dXPRRA_Cnay4unXbYz4nRc1Vn8ubwuvyki5Citk22zbg4bfjfRzJmlIHSrwhJmpZWGqiSdEQ8mP0xyw4ZXBoWoLdFAQ-HChX05RZONDpo6xoeuKWh8i_choOfk9l7Sd2y.png)

## ELMo

ELMo (Peters et al., 2018) es una nueva técnica para calcular y asignar vectores de palabras de manera dinámica.

Los vectores que asigna ELMo a las palabras cambian dependiendo del contexto porque tiene en cuenta todo el texto.

El algoritmo combina dos modelos de lenguaje para calcular probabilidades de una palabra considerando su contexto anterior y posterior.

## BERT

BERT (Devlin et al., 2018) es otro algoritmo para codificar texto como vectores dinámicos.

Combina varias ideas de ELMo (modelos de lenguaje bidireccionales) y otras arquitecturas de DL ([transformers](http://www.peterbloem.nl/blog/transformers)) para crear un modelo generalista que funciona sorprendentemente bien en cualquier tarea y es muy fácil de ajustar para tareas concretas.

### Pero es que hay más

Por fin contamos con modelos de lenguaje pre-entrenados que pueden ser ajustados (*fine tuned*) para tareas concretas.

- [ULMFiT](http://nlp.fast.ai/classification/2018/05/15/introducing-ulmfit.html)
- [OpenAI GPT-2](https://openai.com/blog/better-language-models/)
- [XLNet](https://towardsdatascience.com/what-is-xlnet-and-why-it-outperforms-bert-8d8fce710335)
- [RoBERTa](https://ai.facebook.com/blog/roberta-an-optimized-method-for-pretraining-self-supervised-nlp-systems/)
- [ALBERT](https://medium.com/@lessw/meet-albert-a-new-lite-bert-from-google-toyota-with-state-of-the-art-nlp-performance-and-18x-df8f7b58fa28)


# Modelos de Lenguaje de OpenAI

A mitad de febrero, [OpenAI publicó un modelo de lenguaje](https://blog.openai.com/better-language-models/) capaz de generar lenguaje natural de formar coherente. Este modelo es generalista y, a pesar de ello, es capaz de rivalizar con los mejores sistemas específicos en tareas como comprensión automática de lenguaje natural, traducción automática, búsqueda de respuestas y resumen automático.

Este modelo, llamado GPT-2, es el resultado de haber entrenado con 8 millones de páginas web (40 GB) con 1 500 millones de parámetros con un único objetivo: predecir cuál es la siguiente palabra.

Sin embargo, OpenAI no ha publicado [hasta justo esta semana](https://twitter.com/OpenAI/status/1191764001434173440) el modelo más grande y potente que tenían, para evitar que alguien con malas intenciones pueda hacer un uso dañino de esta tecnología. Sí que han publicado una versión simplificada y más pequeña, y el paper ["Language Models are Unsupervised Multitask Learners"](https://d4mucfpksywv.cloudfront.net/better-language-models/language_models_are_unsupervised_multitask_learners.pdf), en el que explican todo el proceso.

Con ganas y GPUs suficientes (+ tiempo y dinero), se puede replicar el proceso. Otras lecturas interesantes, sobre el tema: 

- [OpenAI's new Multitalented AI Writes, Translates, and Slanders](https://www.theverge.com/2019/2/14/18224704/ai-machine-learning-language-models-read-write-openai-gpt2)
- [Some thoughts on zero-day threats in AI, and OpenAI's GP2](https://www.fast.ai/2019/02/15/openai-gp2/)


Este código de ejemplo está inspirado en [un tweet de Thomas Wolf](https://twitter.com/Thom_Wolf/status/1097465312579072000), de [Hugging Face](https://huggingface.co/).

In [ ]:
import torch 
from torch.nn import functional as F 
from pytorch_transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

A continuación, definimos una función para:

1. tokenizar el texto de entrada y codificarlo como un vector con los pesos obtenidos por el modelo GPT2
2. predecir la siguiente palabra más frecuente
3. decodificar el vector como una secuencia de tokens

In [ ]:
def generate(text, length=50):
    """Generate automatic Natural Language from the input text"""
    vec_text = tokenizer.encode(text)
    my_input, past = torch.tensor([vec_text]), None
    
    for _ in range(length):
        logits, past = model(my_input, past=past)
        my_input = torch.multinomial(F.softmax(logits[:, -1]), 1)
        vec_text.append(my_input.item())
    
    return tokenizer.decode(vec_text)

In [ ]:
# defino un texto de entrada
text = "The only think we can do to fight climate change is"

# y generamos automáticamente las secuencias más probables
print(generate(text, 25))